In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
import requests
from urllib.parse import urlparse
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'

# Read Data

In [2]:
main_data_df = pd.read_csv('main_data_df_20201224-130525.csv')
main_data_df.shape

(4667685, 21)

In [3]:
main_data_df.head()

,시군구,번지,본번,부번,건물명,전용면적(㎡),계약년월,계약일,가격(만원),층,건축년도,도로명,가격/면적,year,month,시,구,전월세매매구분,건물종류,지번주소,도로명주소
0,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201309,8,57000,2,1988,언주로 103,733.118971,2013,9,서울특별시,강남구,매매,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0
1,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201312,16,57000,2,1988,언주로 103,733.118971,2013,12,서울특별시,강남구,매매,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0
2,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,67.28,201302,11,55000,5,1987,언주로 3,817.479191,2013,2,서울특별시,강남구,매매,아파트,서울특별시 강남구 개포동 658-1,서울특별시 강남구 언주로 3-0
3,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,67.28,201302,22,58250,4,1987,언주로 3,865.784780,2013,2,서울특별시,강남구,매매,아파트,서울특별시 강남구 개포동 658-1,서울특별시 강남구 언주로 3-0
4,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,67.28,201305,10,60000,5,1987,언주로 3,891.795482,2013,5,서울특별시,강남구,매매,아파트,서울특별시 강남구 개포동 658-1,서울특별시 강남구 언주로 3-0


In [4]:
main_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4667685 entries, 0 to 4667684
Data columns (total 21 columns):
 #   Column   Dtype  
---  ------   -----  
 0   시군구      object 
 1   번지       object 
 2   본번       int64  
 3   부번       int64  
 4   건물명      object 
 5   전용면적(㎡)  float64
 6   계약년월     int64  
 7   계약일      int64  
 8   가격(만원)   int64  
 9   층        int64  
 10  건축년도     int64  
 11  도로명      object 
 12  가격/면적    float64
 13  year     int64  
 14  month    int64  
 15  시        object 
 16  구        object 
 17  전월세매매구분  object 
 18  건물종류     object 
 19  지번주소     object 
 20  도로명주소    object 
dtypes: float64(2), int64(9), object(10)
memory usage: 747.8+ MB


In [42]:
from_2013_df = main_data_df[main_data_df['year'] >= 2013]
from_2013_df.shape

(3378284, 21)

In [43]:
from_2013_df.head()

,시군구,번지,본번,부번,건물명,전용면적(㎡),계약년월,계약일,가격(만원),층,건축년도,도로명,가격/면적,year,month,시,구,전월세매매구분,건물종류,지번주소,도로명주소
0,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201309,8,57000,2,1988,언주로 103,733.118971,2013,9,서울특별시,강남구,매매,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0
1,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201312,16,57000,2,1988,언주로 103,733.118971,2013,12,서울특별시,강남구,매매,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0
2,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,67.28,201302,11,55000,5,1987,언주로 3,817.479191,2013,2,서울특별시,강남구,매매,아파트,서울특별시 강남구 개포동 658-1,서울특별시 강남구 언주로 3-0
3,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,67.28,201302,22,58250,4,1987,언주로 3,865.784780,2013,2,서울특별시,강남구,매매,아파트,서울특별시 강남구 개포동 658-1,서울특별시 강남구 언주로 3-0
4,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,67.28,201305,10,60000,5,1987,언주로 3,891.795482,2013,5,서울특별시,강남구,매매,아파트,서울특별시 강남구 개포동 658-1,서울특별시 강남구 언주로 3-0


In [83]:
def landnum_modifier(x):
    # x is a string
    
    splitted = x.split('-')
    if len(splitted) == 1:
        return x + '-0'
    else:
        return x

def create_land_specs_df():
    df = pd.read_csv('./토지특성정보/AL_11_D195_20200925.csv', encoding='euc-kr', usecols=[2, 4, 5, 7, 8, 10, 11, 13, 17, 19, 21, 23])
    
    house_list = ['단독, 다세대', '주거기타', '아파트', '연립']
    
    df = df[df['대장구분명'] == '일반']
    df = df[df['토지이동상황'].isin(house_list)]
    df = df[df['기준월'] == 1].reset_index(drop=True)
    
    df.fillna('지정되지않음', inplace=True)
    
    df['전체주소'] = df['법정동명'] + ' ' + df['지번'].apply(landnum_modifier)
    
    categorical_cols = ['지목명', '용도지역명1', '지형높이', '지형형상', '도로접면']
    
    for col in tqdm(categorical_cols, position=0):
        unique_values = df[col].unique()
        for v in unique_values:
            df[col + '_' + '{}'.format(v)] = 0
            df[col + '_' + '{}'.format(v)] = df[col].apply(lambda x: 1 if x == v else 0)
    
    df.drop(columns=categorical_cols + ['법정동명', '지번', '토지이동상황', '기준월', '대장구분명'], inplace=True)
    
    return df

In [84]:
land_specs_df = create_land_specs_df()

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:09<00:00,  1.92s/it]


In [85]:
#land_specs_df = pd.read_csv('land_specs_df_20201228-104930.csv')
land_specs_df.shape

(537395, 64)

In [86]:
land_specs_df.rename(columns={'기준년도':'year', '전체주소':'지번주소'}, inplace=True)

In [87]:
land_specs_df.head()

,year,토지면적,지번주소,지목명_대,지목명_임야,지목명_도로,지목명_학교용지,지목명_종교용지,지목명_사적지,지목명_주차장,지목명_하천,지목명_구거,지목명_수도용지,지목명_공원,지목명_잡종지,지목명_전,지목명_철도용지,지목명_창고용지,지목명_주유소용지,지목명_답,지목명_유지,지목명_공장용지,지목명_제방,지목명_묘지,지목명_체육용지,지목명_과수원,용도지역명1_제1종일반주거지역,용도지역명1_자연녹지지역,용도지역명1_제2종일반주거지역,용도지역명1_제3종일반주거지역,용도지역명1_일반상업지역,용도지역명1_준주거지역,용도지역명1_제1종전용주거지역,용도지역명1_개발제한구역,용도지역명1_보전녹지지역,용도지역명1_준공업지역,용도지역명1_근린상업지역,용도지역명1_제2종전용주거지역,용도지역명1_생산녹지지역,지형높이_평지,지형높이_완경사,지형높이_고지,지형높이_급경사,지형높이_저지,지형형상_사다리형,지형형상_부정형,지형형상_가로장방,지형형상_삼각형,지형형상_세로장방,지형형상_자루형,지형형상_정방형,지형형상_역삼각형,도로접면_소로한면,도로접면_소로각지,도로접면_세로한면(가),도로접면_세로각지(가),도로접면_광대로한면,도로접면_중로한면,도로접면_중로각지,도로접면_세로한면(불),도로접면_광대소각,도로접면_광대세각,도로접면_세로각지(불),도로접면_맹지
0,2013,15622.1,서울특별시 종로구 청운동 1-0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,2014,15622.1,서울특별시 종로구 청운동 1-0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,2015,15622.1,서울특별시 종로구 청운동 1-0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,2016,15622.1,서울특별시 종로구 청운동 1-0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,2017,15622.1,서울특별시 종로구 청운동 1-0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [88]:
land_specs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 537395 entries, 0 to 537394
Data columns (total 64 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   year              537395 non-null  int64  
 1   토지면적              537395 non-null  float64
 2   지번주소              537395 non-null  object 
 3   지목명_대             537395 non-null  int64  
 4   지목명_임야            537395 non-null  int64  
 5   지목명_도로            537395 non-null  int64  
 6   지목명_학교용지          537395 non-null  int64  
 7   지목명_종교용지          537395 non-null  int64  
 8   지목명_사적지           537395 non-null  int64  
 9   지목명_주차장           537395 non-null  int64  
 10  지목명_하천            537395 non-null  int64  
 11  지목명_구거            537395 non-null  int64  
 12  지목명_수도용지          537395 non-null  int64  
 13  지목명_공원            537395 non-null  int64  
 14  지목명_잡종지           537395 non-null  int64  
 15  지목명_전             537395 non-null  int64  
 16  지목명_철도용지          53

In [14]:
land_prices_df = pd.read_csv('land_prices_df_20201224-130609.csv')
land_prices_df.shape

(29393777, 3)

In [15]:
land_prices_df.columns = ['지번주소', 'year', '공시지가(만원)']

In [16]:
land_prices_df.head()

,지번주소,year,공시지가(만원)
0,서울특별시 종로구 청운동 1-0,1990,75.0
1,서울특별시 종로구 청운동 1-0,1991,98.0
2,서울특별시 종로구 청운동 1-0,1992,98.0
3,서울특별시 종로구 청운동 1-0,1993,98.0
4,서울특별시 종로구 청운동 1-0,1994,96.0


In [17]:
land_prices_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29393777 entries, 0 to 29393776
Data columns (total 3 columns):
 #   Column    Dtype  
---  ------    -----  
 0   지번주소      object 
 1   year      int64  
 2   공시지가(만원)  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 672.8+ MB


# EDA & Preprocessing 6

# Merging DFs

In [92]:
print(main_data_df.shape)
print(land_specs_df.shape)
print(land_prices_df.shape)

(4667685, 21)
(537395, 64)
(29393777, 3)


In [19]:
prac_df = main_data_df.merge(land_prices_df, on=['지번주소', 'year'])
prac_df.shape

(4564090, 22)

In [91]:
prac_df.shape

(4564090, 22)

In [90]:
prac_df.head()

,시군구,번지,본번,부번,건물명,전용면적(㎡),계약년월,계약일,가격(만원),층,건축년도,도로명,가격/면적,year,month,시,구,전월세매매구분,건물종류,지번주소,도로명주소,공시지가(만원)
0,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201309,8,57000,2,1988,언주로 103,733.118971,2013,9,서울특별시,강남구,매매,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,921.0
1,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201312,16,57000,2,1988,언주로 103,733.118971,2013,12,서울특별시,강남구,매매,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,921.0
2,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201301,11,35000,4,1988,언주로 103,450.160772,2013,1,서울특별시,강남구,전세,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,921.0
3,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201303,4,32000,9,1988,언주로 103,411.575563,2013,3,서울특별시,강남구,전세,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,921.0
4,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201309,14,37000,1,1988,언주로 103,475.884244,2013,9,서울특별시,강남구,전세,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,921.0


In [63]:
land_specs_df.head()

,year,토지면적,지번주소,지목명_대,지목명_임야,지목명_도로,지목명_학교용지,지목명_종교용지,지목명_사적지,지목명_주차장,지목명_하천,지목명_구거,지목명_수도용지,지목명_공원,지목명_잡종지,지목명_전,지목명_철도용지,지목명_창고용지,지목명_주유소용지,지목명_답,지목명_유지,지목명_묘지,지목명_공장용지,지목명_제방,지목명_체육용지,지목명_과수원,용도지역명1_제1종일반주거지역,용도지역명1_자연녹지지역,용도지역명1_제2종일반주거지역,용도지역명1_제3종일반주거지역,용도지역명1_일반상업지역,용도지역명1_준주거지역,용도지역명1_제1종전용주거지역,용도지역명1_개발제한구역,용도지역명1_보전녹지지역,용도지역명1_준공업지역,용도지역명1_근린상업지역,용도지역명1_제2종전용주거지역,용도지역명1_생산녹지지역,지형높이_평지,지형높이_완경사,지형높이_고지,지형높이_급경사,지형높이_저지,지형형상_사다리형,지형형상_부정형,지형형상_가로장방,지형형상_삼각형,지형형상_세로장방,지형형상_자루형,지형형상_정방형,지형형상_역삼각형,도로접면_소로한면,도로접면_소로각지,도로접면_세로한면(가),도로접면_세로각지(가),도로접면_광대로한면,도로접면_중로한면,도로접면_중로각지,도로접면_세로한면(불),도로접면_광대소각,도로접면_광대세각,도로접면_세로각지(불),도로접면_맹지
0,2013,15622.1,서울특별시 종로구 청운동 1-0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,2014,15622.1,서울특별시 종로구 청운동 1-0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,2015,15622.1,서울특별시 종로구 청운동 1-0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,2016,15622.1,서울특별시 종로구 청운동 1-0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,2017,15622.1,서울특별시 종로구 청운동 1-0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [66]:
original_land_specs_df.head()

,고유번호,법정동코드,법정동명,대장구분코드,대장구분명,지번,토지일련번호,기준년도,기준월,지목코드,지목명,토지면적,용도지역코드1,용도지역명1,용도지역코드2,용도지역명2,토지이용상황코드,토지이동상황,지형높이코드,지형높이,지형형상코드,지형형상,도로접면코드,도로접면,공시지가,데이터기준일자
0,1111010100100010000,1111010100,서울특별시 종로구 청운동,1.0,일반,1,1,2013,1,8,대,15622.1,13,제1종일반주거지역,0,지정되지않음,120,연립,2,평지,4,사다리형,6,소로한면,2840000,2017-10-17
1,1111010100100010000,1111010100,서울특별시 종로구 청운동,1.0,일반,1,1,2014,1,8,대,15622.1,13,제1종일반주거지역,0,지정되지않음,140,아파트,2,평지,4,사다리형,6,소로한면,2933000,2017-10-17
2,1111010100100010000,1111010100,서울특별시 종로구 청운동,1.0,일반,1,1,2015,1,8,대,15622.1,13,제1종일반주거지역,0,지정되지않음,140,아파트,2,평지,4,사다리형,6,소로한면,3053000,2017-10-17
3,1111010100100010000,1111010100,서울특별시 종로구 청운동,1.0,일반,1,1,2016,1,8,대,15622.1,13,제1종일반주거지역,0,지정되지않음,140,아파트,2,평지,4,사다리형,6,소로한면,3234000,2017-10-17
4,1111010100100010000,1111010100,서울특별시 종로구 청운동,1.0,일반,1,1,2017,1,8,대,15622.1,13,제1종일반주거지역,0,지정되지않음,140,아파트,2,평지,4,사다리형,6,소로한면,3488000,2018-05-23


In [71]:
inv_df = original_land_specs_df[(original_land_specs_df['법정동명'] == '서울특별시 종로구 청운동')
                                & (original_land_specs_df['지번'] == '56-113')]
inv_df.shape

(9, 26)

In [73]:
inv_df

,고유번호,법정동코드,법정동명,대장구분코드,대장구분명,지번,토지일련번호,기준년도,기준월,지목코드,지목명,토지면적,용도지역코드1,용도지역명1,용도지역코드2,용도지역명2,토지이용상황코드,토지이동상황,지형높이코드,지형높이,지형형상코드,지형형상,도로접면코드,도로접면,공시지가,데이터기준일자
3925,1111010100100560113,1111010100,서울특별시 종로구 청운동,1.0,일반,56-113,468,2013,1,5,임야,41.3,13,제1종일반주거지역,0,지정되지않음,160,주거기타,4,급경사,7,부정형,10,세로한면(불),1990000,2017-10-17
3926,1111010100100560113,1111010100,서울특별시 종로구 청운동,1.0,일반,56-113,467,2014,1,5,임야,41.3,13,제1종일반주거지역,0,지정되지않음,160,주거기타,4,급경사,7,부정형,10,세로한면(불),2031000,2017-10-17
3927,1111010100100560113,1111010100,서울특별시 종로구 청운동,1.0,일반,56-113,233,2014,7,5,임야,26.7,13,제1종일반주거지역,0,지정되지않음,160,주거기타,4,급경사,7,부정형,10,세로한면(불),2093000,2017-10-17
3928,1111010100100560113,1111010100,서울특별시 종로구 청운동,1.0,일반,56-113,466,2015,1,5,임야,26.7,13,제1종일반주거지역,0,지정되지않음,160,주거기타,4,급경사,7,부정형,10,세로한면(불),2117000,2017-10-17
3929,1111010100100560113,1111010100,서울특별시 종로구 청운동,1.0,일반,56-113,466,2016,1,5,임야,26.7,13,제1종일반주거지역,0,지정되지않음,160,주거기타,4,급경사,7,부정형,10,세로한면(불),2241000,2017-10-17
3930,1111010100100560113,1111010100,서울특별시 종로구 청운동,1.0,일반,56-113,467,2017,1,5,임야,26.7,13,제1종일반주거지역,0,지정되지않음,160,주거기타,4,급경사,7,부정형,10,세로한면(불),2423000,2018-05-23
3931,1111010100100560113,1111010100,서울특별시 종로구 청운동,1.0,일반,56-113,474,2018,1,5,임야,26.7,13,제1종일반주거지역,0,지정되지않음,160,주거기타,4,급경사,7,부정형,10,세로한면(불),2687000,2018-07-31
3932,1111010100100560113,1111010100,서울특별시 종로구 청운동,1.0,일반,56-113,478,2019,1,5,임야,26.7,13,제1종일반주거지역,0,지정되지않음,160,주거기타,4,급경사,7,부정형,10,세로한면(불),2877000,2019-08-08
3933,1111010100100560113,1111010100,서울특별시 종로구 청운동,1.0,일반,56-113,478,2020,1,5,임야,26.7,13,제1종일반주거지역,0,지정되지않음,160,주거기타,4,급경사,7,부정형,10,세로한면(불),2977000,2020-08-12


In [81]:
inv_df = original_land_specs_df[(original_land_specs_df['법정동명'] == '서울특별시 종로구 계동') & 
                                (original_land_specs_df['지번'] == '1-1')]
inv_df.shape

(16, 26)

In [82]:
inv_df

,고유번호,법정동코드,법정동명,대장구분코드,대장구분명,지번,토지일련번호,기준년도,기준월,지목코드,지목명,토지면적,용도지역코드1,용도지역명1,용도지역코드2,용도지역명2,토지이용상황코드,토지이동상황,지형높이코드,지형높이,지형형상코드,지형형상,도로접면코드,도로접면,공시지가,데이터기준일자
114674,1111014800100010001,1111014800,서울특별시 종로구 계동,1.0,일반,1-1,14122,2013,1,8,대,20.8,13,제1종일반주거지역,0,지정되지않음,120,연립,3,완경사,7,부정형,4,중로한면,3620000,2017-10-17
114675,1111014800100010001,1111014800,서울특별시 종로구 계동,1.0,일반,1-1,13981,2014,1,8,대,20.8,13,제1종일반주거지역,0,지정되지않음,120,연립,3,완경사,7,부정형,4,중로한면,3768000,2017-10-17
114676,1111014800100010001,1111014800,서울특별시 종로구 계동,1.0,일반,1-1,13779,2015,1,8,대,20.8,13,제1종일반주거지역,0,지정되지않음,120,연립,3,완경사,7,부정형,4,중로한면,4040000,2017-10-17
114677,1111014800100010001,1111014800,서울특별시 종로구 계동,1.0,일반,1-1,13669,2016,1,8,대,20.8,13,제1종일반주거지역,0,지정되지않음,120,연립,3,완경사,7,부정형,4,중로한면,4176000,2017-10-17
114678,1111014800100010001,1111014800,서울특별시 종로구 계동,1.0,일반,1-1,13540,2017,1,8,대,20.8,13,제1종일반주거지역,0,지정되지않음,120,연립,3,완경사,7,부정형,4,중로한면,4671000,2018-05-23
114679,1111014800100010001,1111014800,서울특별시 종로구 계동,1.0,일반,1-1,13468,2018,1,8,대,20.8,13,제1종일반주거지역,0,지정되지않음,120,연립,3,완경사,7,부정형,4,중로한면,4859000,2018-07-31
114680,1111014800100010001,1111014800,서울특별시 종로구 계동,1.0,일반,1-1,13312,2019,1,8,대,20.8,13,제1종일반주거지역,0,지정되지않음,120,연립,3,완경사,7,부정형,4,중로한면,5002000,2019-08-08
114681,1111014800100010001,1111014800,서울특별시 종로구 계동,1.0,일반,1-1,13350,2020,1,8,대,20.8,13,제1종일반주거지역,0,지정되지않음,120,연립,3,완경사,7,부정형,4,중로한면,5352000,2020-08-12
120007,1111014800200010001,1111014800,서울특별시 종로구 계동,2.0,산,1-1,14769,2013,1,5,임야,14969.0,43,자연녹지지역,0,지정되지않음,160,주거기타,4,급경사,7,부정형,8,세로한면(가),827000,2017-10-17
120008,1111014800200010001,1111014800,서울특별시 종로구 계동,2.0,산,1-1,14628,2014,1,5,임야,14969.0,43,자연녹지지역,0,지정되지않음,160,주거기타,4,급경사,7,부정형,8,세로한면(가),838700,2017-10-17


In [89]:
unique_addrs = land_specs_df['지번주소'].unique()

count = 0
for addr in tqdm(unique_addrs, position=0):
    addr_df = land_specs_df[land_specs_df['지번주소'] == addr]
    unique_years = addr_df['year'].unique()
    for year in unique_years:
        year_df = addr_df[addr_df['year'] == year]
        if year_df.shape[0] > 1:
            count += 1
            display(year_df)            
            raise Exception('my error')

 16%|████████████▎                                                              | 17562/107045 [09:02<46:03, 32.38it/s]


KeyboardInterrupt: 

In [28]:
original_land_specs_df = pd.read_csv('./토지특성정보/AL_11_D195_20200925.csv', encoding='euc-kr')
original_land_specs_df.shape

C:\Users\나\AppData\Roaming\Python\Python37\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(7468497, 26)

In [29]:
original_land_specs_df.head()

,고유번호,법정동코드,법정동명,대장구분코드,대장구분명,지번,토지일련번호,기준년도,기준월,지목코드,지목명,토지면적,용도지역코드1,용도지역명1,용도지역코드2,용도지역명2,토지이용상황코드,토지이동상황,지형높이코드,지형높이,지형형상코드,지형형상,도로접면코드,도로접면,공시지가,데이터기준일자
0,1111010100100010000,1111010100,서울특별시 종로구 청운동,1.0,일반,1,1,2013,1,8,대,15622.1,13,제1종일반주거지역,0,지정되지않음,120,연립,2,평지,4,사다리형,6,소로한면,2840000,2017-10-17
1,1111010100100010000,1111010100,서울특별시 종로구 청운동,1.0,일반,1,1,2014,1,8,대,15622.1,13,제1종일반주거지역,0,지정되지않음,140,아파트,2,평지,4,사다리형,6,소로한면,2933000,2017-10-17
2,1111010100100010000,1111010100,서울특별시 종로구 청운동,1.0,일반,1,1,2015,1,8,대,15622.1,13,제1종일반주거지역,0,지정되지않음,140,아파트,2,평지,4,사다리형,6,소로한면,3053000,2017-10-17
3,1111010100100010000,1111010100,서울특별시 종로구 청운동,1.0,일반,1,1,2016,1,8,대,15622.1,13,제1종일반주거지역,0,지정되지않음,140,아파트,2,평지,4,사다리형,6,소로한면,3234000,2017-10-17
4,1111010100100010000,1111010100,서울특별시 종로구 청운동,1.0,일반,1,1,2017,1,8,대,15622.1,13,제1종일반주거지역,0,지정되지않음,140,아파트,2,평지,4,사다리형,6,소로한면,3488000,2018-05-23


In [38]:
original_land_specs_df['지목명'].value_counts()

대        5712268
도로       1072516
임야        156747
전         115398
답          69480
하천         64604
잡종지        57567
구거         46950
공원         44667
철도용지       32830
학교용지       23863
종교용지       16273
제방         13728
공장용지       10553
주차장         8065
주유소용지       7017
수도용지        4989
사적지         3156
묘지          2927
유지          2303
과수원          973
체육용지         717
창고용지         709
유원지          189
목장용지           8
Name: 지목명, dtype: int64

In [33]:
original_land_specs_df['대장구분명'].value_counts()

일반             7309772
산               126189
가지번              13550
블럭지번(지구)          9996
블럭지번              4824
기타지번              2830
블럭지번(지구_롯트)       1030
가지번(부번세분)          304
Name: 대장구분명, dtype: int64

In [50]:
original_land_specs_df['토지이동상황'].value_counts()

단독          2304087
도로등         1409508
상업용          735177
다세대          689156
주상용          568388
주거기타         305141
주거나지         246717
주상기타         203463
아파트          162714
상업기타         156126
업무용           97038
자연림           95197
전             79725
연립            79257
하천등           69911
공원등           63492
상업나지          39704
공업용           32485
전기타           25761
주상나지          22135
토지임야          20360
공업기타           9086
주차장등           8982
답              7482
운동장등           6832
기타             5555
유해.혐오시설        5094
임야기타           4808
공업나지           3121
답기타            2947
골프장 대중제        2404
과수원            1572
조림             1447
공항             1297
위험시설           1014
특수기타            276
콘도미니엄           272
발전소             241
여객자동차터미널        142
고속도로휴게소          49
유원지               8
주.상복합용            1
Name: 토지이동상황, dtype: int64

In [45]:
house_list = ['단독, 다세대', '주거기타', '아파트', '연립']

In [47]:
house_land_specs_df = original_land_specs_df[original_land_specs_df['토지이동상황'].isin(house_list)]
house_land_specs_df.shape

(547112, 26)

In [49]:
house_land_specs_df.head()

,고유번호,법정동코드,법정동명,대장구분코드,대장구분명,지번,토지일련번호,기준년도,기준월,지목코드,지목명,토지면적,용도지역코드1,용도지역명1,용도지역코드2,용도지역명2,토지이용상황코드,토지이동상황,지형높이코드,지형높이,지형형상코드,지형형상,도로접면코드,도로접면,공시지가,데이터기준일자
0,1111010100100010000,1111010100,서울특별시 종로구 청운동,1.0,일반,1,1,2013,1,8,대,15622.1,13,제1종일반주거지역,0,지정되지않음,120,연립,2,평지,4,사다리형,6,소로한면,2840000,2017-10-17
1,1111010100100010000,1111010100,서울특별시 종로구 청운동,1.0,일반,1,1,2014,1,8,대,15622.1,13,제1종일반주거지역,0,지정되지않음,140,아파트,2,평지,4,사다리형,6,소로한면,2933000,2017-10-17
2,1111010100100010000,1111010100,서울특별시 종로구 청운동,1.0,일반,1,1,2015,1,8,대,15622.1,13,제1종일반주거지역,0,지정되지않음,140,아파트,2,평지,4,사다리형,6,소로한면,3053000,2017-10-17
3,1111010100100010000,1111010100,서울특별시 종로구 청운동,1.0,일반,1,1,2016,1,8,대,15622.1,13,제1종일반주거지역,0,지정되지않음,140,아파트,2,평지,4,사다리형,6,소로한면,3234000,2017-10-17
4,1111010100100010000,1111010100,서울특별시 종로구 청운동,1.0,일반,1,1,2017,1,8,대,15622.1,13,제1종일반주거지역,0,지정되지않음,140,아파트,2,평지,4,사다리형,6,소로한면,3488000,2018-05-23


In [44]:
original_land_specs_df['토지이동상황'].value_counts()

단독          2304087
도로등         1409508
상업용          735177
다세대          689156
주상용          568388
주거기타         305141
주거나지         246717
주상기타         203463
아파트          162714
상업기타         156126
업무용           97038
자연림           95197
전             79725
연립            79257
하천등           69911
공원등           63492
상업나지          39704
공업용           32485
전기타           25761
주상나지          22135
토지임야          20360
공업기타           9086
주차장등           8982
답              7482
운동장등           6832
기타             5555
유해.혐오시설        5094
임야기타           4808
공업나지           3121
답기타            2947
골프장 대중제        2404
과수원            1572
조림             1447
공항             1297
위험시설           1014
특수기타            276
콘도미니엄           272
발전소             241
여객자동차터미널        142
고속도로휴게소          49
유원지               8
주.상복합용            1
Name: 토지이동상황, dtype: int64

In [36]:
mount_df = original_land_specs_df[original_land_specs_df['대장구분명'] == '산']
mount_df.shape

(126189, 26)

In [37]:
mount_df.head()

,고유번호,법정동코드,법정동명,대장구분코드,대장구분명,지번,토지일련번호,기준년도,기준월,지목코드,지목명,토지면적,용도지역코드1,용도지역명1,용도지역코드2,용도지역명2,토지이용상황코드,토지이동상황,지형높이코드,지형높이,지형형상코드,지형형상,도로접면코드,도로접면,공시지가,데이터기준일자
5736,1111010100200010001,1111010100,서울특별시 종로구 청운동,2.0,산,1-1,684,2013,1,5,임야,254940.0,44,개발제한구역,0,지정되지않음,720,자연림,5,고지,7,부정형,4,중로한면,40000,2017-10-17
5737,1111010100200010001,1111010100,서울특별시 종로구 청운동,2.0,산,1-1,680,2014,1,5,임야,254940.0,44,개발제한구역,0,지정되지않음,720,자연림,5,고지,7,부정형,4,중로한면,40800,2017-10-17
5738,1111010100200010001,1111010100,서울특별시 종로구 청운동,2.0,산,1-1,680,2015,1,5,임야,254940.0,44,개발제한구역,0,지정되지않음,720,자연림,5,고지,7,부정형,4,중로한면,42400,2017-10-17
5739,1111010100200010001,1111010100,서울특별시 종로구 청운동,2.0,산,1-1,683,2016,1,5,임야,254940.0,44,개발제한구역,0,지정되지않음,720,자연림,5,고지,7,부정형,4,중로한면,43200,2017-10-17
5740,1111010100200010001,1111010100,서울특별시 종로구 청운동,2.0,산,1-1,684,2017,1,5,임야,254940.0,44,개발제한구역,13,제1종일반주거지역,720,자연림,5,고지,7,부정형,4,중로한면,58700,2018-05-23


In [39]:
mount_df.tail()

,고유번호,법정동코드,법정동명,대장구분코드,대장구분명,지번,토지일련번호,기준년도,기준월,지목코드,지목명,토지면적,용도지역코드1,용도지역명1,용도지역코드2,용도지역명2,토지이용상황코드,토지이동상황,지형높이코드,지형높이,지형형상코드,지형형상,도로접면코드,도로접면,공시지가,데이터기준일자
7467373,1174011000200260000,1174011000,서울특별시 강동구 강일동,2.0,산,26,28912,2016,1,5,임야,1190.0,14,제2종일반주거지역,0,지정되지않음,720,자연림,2,평지,7,부정형,8,세로한면(가),311000,2017-10-17
7467374,1174011000200260000,1174011000,서울특별시 강동구 강일동,2.0,산,26,28955,2017,1,5,임야,1190.0,14,제2종일반주거지역,0,지정되지않음,720,자연림,2,평지,7,부정형,8,세로한면(가),348300,2018-05-23
7467375,1174011000200260000,1174011000,서울특별시 강동구 강일동,2.0,산,26,29007,2018,1,5,임야,1190.0,14,제2종일반주거지역,0,지정되지않음,720,자연림,2,평지,7,부정형,8,세로한면(가),350800,2018-07-31
7467376,1174011000200260000,1174011000,서울특별시 강동구 강일동,2.0,산,26,29139,2019,1,5,임야,1190.0,14,제2종일반주거지역,0,지정되지않음,720,자연림,2,평지,7,부정형,8,세로한면(가),369300,2019-08-08
7467377,1174011000200260000,1174011000,서울특별시 강동구 강일동,2.0,산,26,29340,2020,1,5,임야,1190.0,14,제2종일반주거지역,0,지정되지않음,720,자연림,2,평지,7,부정형,8,세로한면(가),402500,2020-08-12


In [40]:
mount_df['지목명'].value_counts()

임야      110004
도로        9076
공원        2838
구거         992
묘지         771
수도용지       633
철도용지       543
하천         437
학교용지       275
제방         216
종교용지        99
전           98
과수원         86
잡종지         50
대           47
답            9
주차장          9
사적지          6
Name: 지목명, dtype: int64

In [41]:
mount_df['토지이동상황'].value_counts()

자연림        73724
도로등        23713
토지임야        5515
주거기타        4105
공원등         3119
주거나지        2995
임야기타        2906
단독          2245
전기타         1171
아파트          978
상업기타         897
하천등          827
전            633
상업용          588
운동장등         470
다세대          433
과수원          325
조림           272
연립           253
기타           149
업무용          142
상업나지         118
주상용          118
위험시설          96
주상기타          86
주차장등          84
콘도미니엄         40
주상나지          36
유해.혐오시설       30
공업나지          25
답기타           22
공항            21
공업기타          14
공업용           13
골프장 대중제       12
특수기타          10
답              4
Name: 토지이동상황, dtype: int64

In [35]:
original_land_specs_df['토지이동상황'].value_counts()

단독          2304087
도로등         1409508
상업용          735177
다세대          689156
주상용          568388
주거기타         305141
주거나지         246717
주상기타         203463
아파트          162714
상업기타         156126
업무용           97038
자연림           95197
전             79725
연립            79257
하천등           69911
공원등           63492
상업나지          39704
공업용           32485
전기타           25761
주상나지          22135
토지임야          20360
공업기타           9086
주차장등           8982
답              7482
운동장등           6832
기타             5555
유해.혐오시설        5094
임야기타           4808
공업나지           3121
답기타            2947
골프장 대중제        2404
과수원            1572
조림             1447
공항             1297
위험시설           1014
특수기타            276
콘도미니엄           272
발전소             241
여객자동차터미널        142
고속도로휴게소          49
유원지               8
주.상복합용            1
Name: 토지이동상황, dtype: int64

In [94]:
prac_df.shape

(4564090, 22)

In [93]:
prac_df2 = prac_df.merge(land_specs_df, on=['지번주소', 'year'], how='left')
prac_df2.shape

(4564090, 84)

In [95]:
prac_df2.head()

,시군구,번지,본번,부번,건물명,전용면적(㎡),계약년월,계약일,가격(만원),층,건축년도,도로명,가격/면적,year,month,시,구,전월세매매구분,건물종류,지번주소,도로명주소,공시지가(만원),토지면적,지목명_대,지목명_임야,지목명_도로,지목명_학교용지,지목명_종교용지,지목명_사적지,지목명_주차장,지목명_하천,지목명_구거,지목명_수도용지,지목명_공원,지목명_잡종지,지목명_전,지목명_철도용지,지목명_창고용지,지목명_주유소용지,지목명_답,지목명_유지,지목명_공장용지,지목명_제방,지목명_묘지,지목명_체육용지,지목명_과수원,용도지역명1_제1종일반주거지역,용도지역명1_자연녹지지역,용도지역명1_제2종일반주거지역,용도지역명1_제3종일반주거지역,용도지역명1_일반상업지역,용도지역명1_준주거지역,용도지역명1_제1종전용주거지역,용도지역명1_개발제한구역,용도지역명1_보전녹지지역,용도지역명1_준공업지역,용도지역명1_근린상업지역,용도지역명1_제2종전용주거지역,용도지역명1_생산녹지지역,지형높이_평지,지형높이_완경사,지형높이_고지,지형높이_급경사,지형높이_저지,지형형상_사다리형,지형형상_부정형,지형형상_가로장방,지형형상_삼각형,지형형상_세로장방,지형형상_자루형,지형형상_정방형,지형형상_역삼각형,도로접면_소로한면,도로접면_소로각지,도로접면_세로한면(가),도로접면_세로각지(가),도로접면_광대로한면,도로접면_중로한면,도로접면_중로각지,도로접면_세로한면(불),도로접면_광대소각,도로접면_광대세각,도로접면_세로각지(불),도로접면_맹지
0,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201309,8,57000,2,1988,언주로 103,733.118971,2013,9,서울특별시,강남구,매매,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,921.0,1168.3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201312,16,57000,2,1988,언주로 103,733.118971,2013,12,서울특별시,강남구,매매,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,921.0,1168.3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201301,11,35000,4,1988,언주로 103,450.160772,2013,1,서울특별시,강남구,전세,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,921.0,1168.3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201303,4,32000,9,1988,언주로 103,411.575563,2013,3,서울특별시,강남구,전세,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,921.0,1168.3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201309,14,37000,1,1988,언주로 103,475.884244,2013,9,서울특별시,강남구,전세,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,921.0,1168.3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [96]:
prac_df2['토지면적'].isna().sum()

2724018

In [102]:
land_specs_df.shape

(537395, 64)

In [103]:
land_specs_df.head()

,year,토지면적,지번주소,지목명_대,지목명_임야,지목명_도로,지목명_학교용지,지목명_종교용지,지목명_사적지,지목명_주차장,지목명_하천,지목명_구거,지목명_수도용지,지목명_공원,지목명_잡종지,지목명_전,지목명_철도용지,지목명_창고용지,지목명_주유소용지,지목명_답,지목명_유지,지목명_공장용지,지목명_제방,지목명_묘지,지목명_체육용지,지목명_과수원,용도지역명1_제1종일반주거지역,용도지역명1_자연녹지지역,용도지역명1_제2종일반주거지역,용도지역명1_제3종일반주거지역,용도지역명1_일반상업지역,용도지역명1_준주거지역,용도지역명1_제1종전용주거지역,용도지역명1_개발제한구역,용도지역명1_보전녹지지역,용도지역명1_준공업지역,용도지역명1_근린상업지역,용도지역명1_제2종전용주거지역,용도지역명1_생산녹지지역,지형높이_평지,지형높이_완경사,지형높이_고지,지형높이_급경사,지형높이_저지,지형형상_사다리형,지형형상_부정형,지형형상_가로장방,지형형상_삼각형,지형형상_세로장방,지형형상_자루형,지형형상_정방형,지형형상_역삼각형,도로접면_소로한면,도로접면_소로각지,도로접면_세로한면(가),도로접면_세로각지(가),도로접면_광대로한면,도로접면_중로한면,도로접면_중로각지,도로접면_세로한면(불),도로접면_광대소각,도로접면_광대세각,도로접면_세로각지(불),도로접면_맹지
0,2013,15622.1,서울특별시 종로구 청운동 1-0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,2014,15622.1,서울특별시 종로구 청운동 1-0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,2015,15622.1,서울특별시 종로구 청운동 1-0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,2016,15622.1,서울특별시 종로구 청운동 1-0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,2017,15622.1,서울특별시 종로구 청운동 1-0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [105]:
unique_addrs = land_specs_df['지번주소'].unique()

for addr in tqdm(unique_addrs, position=0):
    addr_df = land_specs_df[land_specs_df['지번주소'] == addr]
    min_year = addr_df['year'].min()
    if min_year != 2013:
        display(addr_df)
        raise Exception(min_year)

  0%|                                                                              | 18/107045 [00:00<55:26, 32.17it/s]

,year,토지면적,지번주소,지목명_대,지목명_임야,지목명_도로,지목명_학교용지,지목명_종교용지,지목명_사적지,지목명_주차장,지목명_하천,지목명_구거,지목명_수도용지,지목명_공원,지목명_잡종지,지목명_전,지목명_철도용지,지목명_창고용지,지목명_주유소용지,지목명_답,지목명_유지,지목명_공장용지,지목명_제방,지목명_묘지,지목명_체육용지,지목명_과수원,용도지역명1_제1종일반주거지역,용도지역명1_자연녹지지역,용도지역명1_제2종일반주거지역,용도지역명1_제3종일반주거지역,용도지역명1_일반상업지역,용도지역명1_준주거지역,용도지역명1_제1종전용주거지역,용도지역명1_개발제한구역,용도지역명1_보전녹지지역,용도지역명1_준공업지역,용도지역명1_근린상업지역,용도지역명1_제2종전용주거지역,용도지역명1_생산녹지지역,지형높이_평지,지형높이_완경사,지형높이_고지,지형높이_급경사,지형높이_저지,지형형상_사다리형,지형형상_부정형,지형형상_가로장방,지형형상_삼각형,지형형상_세로장방,지형형상_자루형,지형형상_정방형,지형형상_역삼각형,도로접면_소로한면,도로접면_소로각지,도로접면_세로한면(가),도로접면_세로각지(가),도로접면_광대로한면,도로접면_중로한면,도로접면_중로각지,도로접면_세로한면(불),도로접면_광대소각,도로접면_광대세각,도로접면_세로각지(불),도로접면_맹지
138,2017,40.3,서울특별시 종로구 청운동 4-43,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
139,2018,40.3,서울특별시 종로구 청운동 4-43,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
140,2019,40.3,서울특별시 종로구 청운동 4-43,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
141,2020,40.3,서울특별시 종로구 청운동 4-43,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


  0%|                                                                              | 19/107045 [00:00<59:16, 30.10it/s]


Exception: 2017

In [106]:
original_land_specs_df.head()

,고유번호,법정동코드,법정동명,대장구분코드,대장구분명,지번,토지일련번호,기준년도,기준월,지목코드,지목명,토지면적,용도지역코드1,용도지역명1,용도지역코드2,용도지역명2,토지이용상황코드,토지이동상황,지형높이코드,지형높이,지형형상코드,지형형상,도로접면코드,도로접면,공시지가,데이터기준일자
0,1111010100100010000,1111010100,서울특별시 종로구 청운동,1.0,일반,1,1,2013,1,8,대,15622.1,13,제1종일반주거지역,0,지정되지않음,120,연립,2,평지,4,사다리형,6,소로한면,2840000,2017-10-17
1,1111010100100010000,1111010100,서울특별시 종로구 청운동,1.0,일반,1,1,2014,1,8,대,15622.1,13,제1종일반주거지역,0,지정되지않음,140,아파트,2,평지,4,사다리형,6,소로한면,2933000,2017-10-17
2,1111010100100010000,1111010100,서울특별시 종로구 청운동,1.0,일반,1,1,2015,1,8,대,15622.1,13,제1종일반주거지역,0,지정되지않음,140,아파트,2,평지,4,사다리형,6,소로한면,3053000,2017-10-17
3,1111010100100010000,1111010100,서울특별시 종로구 청운동,1.0,일반,1,1,2016,1,8,대,15622.1,13,제1종일반주거지역,0,지정되지않음,140,아파트,2,평지,4,사다리형,6,소로한면,3234000,2017-10-17
4,1111010100100010000,1111010100,서울특별시 종로구 청운동,1.0,일반,1,1,2017,1,8,대,15622.1,13,제1종일반주거지역,0,지정되지않음,140,아파트,2,평지,4,사다리형,6,소로한면,3488000,2018-05-23


In [107]:
inv_df = original_land_specs_df[(original_land_specs_df['법정동명'] == '서울특별시 종로구 청운동') & 
                                (original_land_specs_df['지번'] == '4-43')]
inv_df.shape

(4, 26)

In [108]:
inv_df

,고유번호,법정동코드,법정동명,대장구분코드,대장구분명,지번,토지일련번호,기준년도,기준월,지목코드,지목명,토지면적,용도지역코드1,용도지역명1,용도지역코드2,용도지역명2,토지이용상황코드,토지이동상황,지형높이코드,지형높이,지형형상코드,지형형상,도로접면코드,도로접면,공시지가,데이터기준일자
739,1111010100100040043,1111010100,서울특별시 종로구 청운동,1.0,일반,4-43,91,2017,1,5,임야,40.3,13,제1종일반주거지역,0,지정되지않음,160,주거기타,2,평지,7,부정형,8,세로한면(가),3317000,2018-05-23
740,1111010100100040043,1111010100,서울특별시 종로구 청운동,1.0,일반,4-43,91,2018,1,5,임야,40.3,13,제1종일반주거지역,0,지정되지않음,160,주거기타,2,평지,7,부정형,8,세로한면(가),3581000,2018-07-31
741,1111010100100040043,1111010100,서울특별시 종로구 청운동,1.0,일반,4-43,91,2019,1,5,임야,40.3,13,제1종일반주거지역,0,지정되지않음,160,주거기타,2,평지,7,부정형,8,세로한면(가),3767000,2019-08-08
742,1111010100100040043,1111010100,서울특별시 종로구 청운동,1.0,일반,4-43,91,2020,1,5,임야,40.3,13,제1종일반주거지역,0,지정되지않음,160,주거기타,2,평지,7,부정형,8,세로한면(가),3902000,2020-08-12


In [109]:
main_data_df.shape

(4667685, 21)

In [110]:
main_data_df.head()

,시군구,번지,본번,부번,건물명,전용면적(㎡),계약년월,계약일,가격(만원),층,건축년도,도로명,가격/면적,year,month,시,구,전월세매매구분,건물종류,지번주소,도로명주소
0,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201309,8,57000,2,1988,언주로 103,733.118971,2013,9,서울특별시,강남구,매매,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0
1,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201312,16,57000,2,1988,언주로 103,733.118971,2013,12,서울특별시,강남구,매매,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0
2,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,67.28,201302,11,55000,5,1987,언주로 3,817.479191,2013,2,서울특별시,강남구,매매,아파트,서울특별시 강남구 개포동 658-1,서울특별시 강남구 언주로 3-0
3,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,67.28,201302,22,58250,4,1987,언주로 3,865.784780,2013,2,서울특별시,강남구,매매,아파트,서울특별시 강남구 개포동 658-1,서울특별시 강남구 언주로 3-0
4,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,67.28,201305,10,60000,5,1987,언주로 3,891.795482,2013,5,서울특별시,강남구,매매,아파트,서울특별시 강남구 개포동 658-1,서울특별시 강남구 언주로 3-0


In [112]:
inv_df2 = main_data_df[(main_data_df['시군구'] == '서울특별시 종로구 청운동') & (main_data_df['본번'] == 4)]
inv_df2.shape

(42, 21)

In [113]:
inv_df2

,시군구,번지,본번,부번,건물명,전용면적(㎡),계약년월,계약일,가격(만원),층,건축년도,도로명,가격/면적,year,month,시,구,전월세매매구분,건물종류,지번주소,도로명주소
1228275,서울특별시 종로구 청운동,4-14,4,14,리뉴빌,147.02,201401,4,81000,-1,2004,자하문로36길 27,550.945450,2014,1,서울특별시,종로구,매매,연립다세대,서울특별시 종로구 청운동 4-14,서울특별시 종로구 자하문로36길 27-0
1282041,서울특별시 종로구 청운동,4-14,4,14,리뉴빌,160.90,201503,19,79000,2,2004,자하문로36길 27,490.988191,2015,3,서울특별시,종로구,매매,연립다세대,서울특별시 종로구 청운동 4-14,서울특별시 종로구 자하문로36길 27-0
1339292,서울특별시 종로구 청운동,4-7,4,7,신구파인힐,215.09,201608,8,190000,2,2000,자하문로36길 22,883.351155,2016,8,서울특별시,종로구,매매,연립다세대,서울특별시 종로구 청운동 4-7,서울특별시 종로구 자하문로36길 22-0
1438353,서울특별시 종로구 청운동,4-35,4,35,청운빌라,144.16,201801,30,73000,3,1984,자하문로36길 17,506.381798,2018,1,서울특별시,종로구,매매,연립다세대,서울특별시 종로구 청운동 4-35,서울특별시 종로구 자하문로36길 17-0
1480484,서울특별시 종로구 청운동,4-7,4,7,신구파인힐,90.72,201912,31,40000,-1,2000,자하문로36길 22,440.917108,2019,12,서울특별시,종로구,매매,연립다세대,서울특별시 종로구 청운동 4-7,서울특별시 종로구 자하문로36길 22-0
1531115,서울특별시 종로구 청운동,4-35,4,35,청운빌라,144.36,202010,16,72000,2,1984,자하문로36길 17,498.753117,2020,10,서울특별시,종로구,매매,연립다세대,서울특별시 종로구 청운동 4-35,서울특별시 종로구 자하문로36길 17-0
1597882,서울특별시 종로구 청운동,4-35,4,35,청운빌라,143.16,200610,6,40500,1,1984,자하문로36길 17,282.900251,2006,10,서울특별시,종로구,매매,연립다세대,서울특별시 종로구 청운동 4-35,서울특별시 종로구 자하문로36길 17-0
1653858,서울특별시 종로구 청운동,4-3,4,3,GRACETUSCANII,244.08,200709,27,235000,-1,2007,자하문로36길 21,962.799082,2007,9,서울특별시,종로구,매매,연립다세대,서울특별시 종로구 청운동 4-3,서울특별시 종로구 자하문로36길 21-0
1653866,서울특별시 종로구 청운동,4-1,4,1,아델하우스,244.72,200711,29,154058,2,2007,자하문로36길 28,629.527623,2007,11,서울특별시,종로구,매매,연립다세대,서울특별시 종로구 청운동 4-1,서울특별시 종로구 자하문로36길 28-0
1653867,서울특별시 종로구 청운동,4-1,4,1,아델하우스,239.03,200712,20,198000,2,2007,자하문로36길 28,828.347906,2007,12,서울특별시,종로구,매매,연립다세대,서울특별시 종로구 청운동 4-1,서울특별시 종로구 자하문로36길 28-0


In [97]:
from_2013_df = prac_df2[prac_df2['year'] >= 2013]
from_2013_df.shape

(3286171, 84)

In [98]:
from_2013_df.head()

,시군구,번지,본번,부번,건물명,전용면적(㎡),계약년월,계약일,가격(만원),층,건축년도,도로명,가격/면적,year,month,시,구,전월세매매구분,건물종류,지번주소,도로명주소,공시지가(만원),토지면적,지목명_대,지목명_임야,지목명_도로,지목명_학교용지,지목명_종교용지,지목명_사적지,지목명_주차장,지목명_하천,지목명_구거,지목명_수도용지,지목명_공원,지목명_잡종지,지목명_전,지목명_철도용지,지목명_창고용지,지목명_주유소용지,지목명_답,지목명_유지,지목명_공장용지,지목명_제방,지목명_묘지,지목명_체육용지,지목명_과수원,용도지역명1_제1종일반주거지역,용도지역명1_자연녹지지역,용도지역명1_제2종일반주거지역,용도지역명1_제3종일반주거지역,용도지역명1_일반상업지역,용도지역명1_준주거지역,용도지역명1_제1종전용주거지역,용도지역명1_개발제한구역,용도지역명1_보전녹지지역,용도지역명1_준공업지역,용도지역명1_근린상업지역,용도지역명1_제2종전용주거지역,용도지역명1_생산녹지지역,지형높이_평지,지형높이_완경사,지형높이_고지,지형높이_급경사,지형높이_저지,지형형상_사다리형,지형형상_부정형,지형형상_가로장방,지형형상_삼각형,지형형상_세로장방,지형형상_자루형,지형형상_정방형,지형형상_역삼각형,도로접면_소로한면,도로접면_소로각지,도로접면_세로한면(가),도로접면_세로각지(가),도로접면_광대로한면,도로접면_중로한면,도로접면_중로각지,도로접면_세로한면(불),도로접면_광대소각,도로접면_광대세각,도로접면_세로각지(불),도로접면_맹지
0,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201309,8,57000,2,1988,언주로 103,733.118971,2013,9,서울특별시,강남구,매매,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,921.0,1168.3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201312,16,57000,2,1988,언주로 103,733.118971,2013,12,서울특별시,강남구,매매,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,921.0,1168.3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201301,11,35000,4,1988,언주로 103,450.160772,2013,1,서울특별시,강남구,전세,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,921.0,1168.3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201303,4,32000,9,1988,언주로 103,411.575563,2013,3,서울특별시,강남구,전세,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,921.0,1168.3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201309,14,37000,1,1988,언주로 103,475.884244,2013,9,서울특별시,강남구,전세,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,921.0,1168.3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [100]:
prac_df3 = prac_df2.dropna()
prac_df3.shape

(1838281, 84)

In [101]:
prac_df3.head()

,시군구,번지,본번,부번,건물명,전용면적(㎡),계약년월,계약일,가격(만원),층,건축년도,도로명,가격/면적,year,month,시,구,전월세매매구분,건물종류,지번주소,도로명주소,공시지가(만원),토지면적,지목명_대,지목명_임야,지목명_도로,지목명_학교용지,지목명_종교용지,지목명_사적지,지목명_주차장,지목명_하천,지목명_구거,지목명_수도용지,지목명_공원,지목명_잡종지,지목명_전,지목명_철도용지,지목명_창고용지,지목명_주유소용지,지목명_답,지목명_유지,지목명_공장용지,지목명_제방,지목명_묘지,지목명_체육용지,지목명_과수원,용도지역명1_제1종일반주거지역,용도지역명1_자연녹지지역,용도지역명1_제2종일반주거지역,용도지역명1_제3종일반주거지역,용도지역명1_일반상업지역,용도지역명1_준주거지역,용도지역명1_제1종전용주거지역,용도지역명1_개발제한구역,용도지역명1_보전녹지지역,용도지역명1_준공업지역,용도지역명1_근린상업지역,용도지역명1_제2종전용주거지역,용도지역명1_생산녹지지역,지형높이_평지,지형높이_완경사,지형높이_고지,지형높이_급경사,지형높이_저지,지형형상_사다리형,지형형상_부정형,지형형상_가로장방,지형형상_삼각형,지형형상_세로장방,지형형상_자루형,지형형상_정방형,지형형상_역삼각형,도로접면_소로한면,도로접면_소로각지,도로접면_세로한면(가),도로접면_세로각지(가),도로접면_광대로한면,도로접면_중로한면,도로접면_중로각지,도로접면_세로한면(불),도로접면_광대소각,도로접면_광대세각,도로접면_세로각지(불),도로접면_맹지
0,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201309,8,57000,2,1988,언주로 103,733.118971,2013,9,서울특별시,강남구,매매,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,921.0,1168.3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201312,16,57000,2,1988,언주로 103,733.118971,2013,12,서울특별시,강남구,매매,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,921.0,1168.3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201301,11,35000,4,1988,언주로 103,450.160772,2013,1,서울특별시,강남구,전세,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,921.0,1168.3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201303,4,32000,9,1988,언주로 103,411.575563,2013,3,서울특별시,강남구,전세,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,921.0,1168.3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201309,14,37000,1,1988,언주로 103,475.884244,2013,9,서울특별시,강남구,전세,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,921.0,1168.3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [114]:
prac_df2.sort_values(['지번주소', 'year'], inplace=True)
prac_df2.shape

MemoryError: Unable to allocate 348. MiB for an array with shape (10, 4564090) and data type object

In [ ]:
prac_df2.head()